In [1]:
!pip install -q geemap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.1 MB/s eta 0:00:00


In [2]:
import ee
import geemap

In [3]:
import ee
ee.Authenticate()

In [4]:
ee.Initialize(project='lips-482716')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [5]:
output_geojson_path = '/content/cachoeira_seca_georeferenced.geojson'
cachoeira_seca_feature = geemap.geojson_to_ee(output_geojson_path)

In [7]:

roi = cachoeira_seca_feature.geometry().bounds()

In [13]:
map = geemap.Map()

# Define a visualization for the feature layer
vis_params_feature = {
        'color': '800080',  # Purple color
        'pointSize': 5,
        'pointShape': 'circle',
        'width': 2           # Line width
    }
vis_params_roi = {
        'color': '800080',  # Purple color
        'pointSize': 5,
        'pointShape': 'rectangle',
        'width': 2           # Line width
    }
# Add layers to the interactive map for display
map.addLayer(cachoeira_seca_feature, vis_params_feature, 'cachoeira_seca KML')
map.addLayer(roi, vis_params_roi, 'cachoeira_seca roi')

map.centerObject(roi, 8)


map

Map(center=[-4.3623527757297635, -54.06354447666809], controls=(WidgetControl(options=['position', 'transparen…